# **Pequena LLM usando LSTM | Projeto geração de texto usando LSTMs**

Esse projeto é um modelo de Linguagem de Grande (LLM) de pequena escala usando uma rede neural LSTM (Long Short-Term Memory). Este modelo é capaz de gerar texto sequencial com base em padrões aprendidos a partir de dados textuais de entrada. Aqui está um resumo do que o modelo faz: <br>

**Pré-processamento de Texto**

O texto é limpo e tokenizado para prepará-lo para o treinamento.
Sequências de tokens são geradas para alimentar o modelo.


<br>**Construção do Modelo**:

O modelo é construído usando a biblioteca Keras com uma camada de Embedding seguida por uma camada LSTM para capturar dependências sequenciais.
Dropout é usado para regularização e uma camada Dense final com ativação softmax gera a previsão da próxima palavra.

<br>**Geração de Texto:**

Uma função para gerar texto com base em um prompt inicial, onde o modelo prevê palavras sucessivas para formar novas sequências de texto.
Este projeto é um ponto de partida para explorar modelos de linguagem menores e pode ser expandido com mais dados e ajustes para alcançar desempenho superior em tarefas de geração de texto.



In [ ]:
# Módulos Keras para construir a LSTM
import keras.utils as ku
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential

# Frameoworks de manipualçai
import pandas as pd
import numpy as np
import string, os
import warnings

# Configura uma semente para gerar números aleatórios de forma reprodutível
from numpy.random import seed
seed(1)

# Controlar a exibição de avisos
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

1. **keras.utils** <br>
Este módulo fornece várias funções utilitárias para facilitar a manipulação de dados e o pré-processamento necessário para o treinamento de redes neurais.


2. **keras.preprocessing.sequence.pad_sequences** <br>
pad_sequences é uma função útil para garantir que todas as sequências de entrada tenham o mesmo comprimento, preenchendo com zeros (ou outro valor) conforme necessário.


3. **keras.layers.Embedding** <br>
A camada de Embedding é crucial para converter tokens de palavras em vetores densos de dimensão fixa, o que é necessário para a entrada na LSTM.


4. **keras.layers.LSTM** <br>
A camada LSTM é o coração do seu modelo sequencial, permitindo que ele aprenda dependências temporais em dados sequenciais.


5. **keras.layers.Dense** <br>
A camada Dense é uma camada totalmente conectada que geralmente é usada para a saída do modelo, onde cada unidade é conectada a todas as unidades na camada anterior.


6. **keras.layers.Dropout** <br>
A camada Dropout é usada para prevenir overfitting durante o treinamento, desligando aleatoriamente uma fração das unidades durante a etapa de treinamento.


7. **keras.preprocessing.text.Tokenizer** <br>
O Tokenizer é usado para converter texto bruto em tokens, que podem então ser convertidos em sequências de inteiros.


8. **keras.callbacks.EarlyStopping** <br>
EarlyStopping é uma técnica de regularização que interrompe o treinamento do modelo quando a métrica monitorada (como a perda de validação) para de melhorar, prevenindo overfitting e economizando tempo de treinamento.


9. **keras.models.Sequential** <br>
O Sequential é um contêiner linear para empilhar camadas do modelo de forma sequencial, do início ao fim.

### **Leitura dos dados**
Carregar o conjunto de dados de manchetes de notícias.<br>
Para os dados usei um conjunto de dados de notícias do New Yor Times.

In [ ]:
# Pasta com os arquivos
Pasta = 'Artigos/'

# Lista para armazenar os arquivos CSV
Machetes_Localizadas = []

# Loop para percorrer as pastas
for nome_arquivo in os.listdir(Pasta):

    # Verificar se o arquivo é um arquivo CSV
    if 'Articles' in nome_arquivo:

        # Ler o arquivo CSV
        df_artigo = pd.read_csv(Pasta + nome_arquivo)

        # Selecionando a coluna Machetes
        Machetes_Localizadas.extend( list(df_artigo.headline.values) )
        break

# Retirando manchtes com o tema 'Unknown | Desconhecimento'
Machetes_Localizadas = [Loop for Loop in Machetes_Localizadas if Loop != 'Unknown']

print( f'Localizadas {len(Machetes_Localizadas)} manchetes' )

Localizadas 1250 manchetes


In [39]:
# DF dos artigos
df_artigo.head()

,articleID,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL,articleWordCount
0,5a974697410cf7000162e8a4,By BINYAMIN APPELBAUM,article,"Virtual Coins, Real Resources","['Bitcoin (Currency)', 'Electric Light and Pow...",1,Business,1,2018-03-01 00:17:22,Economy,America has a productivity problem. One explan...,The New York Times,News,https://www.nytimes.com/2018/02/28/business/ec...,1207
1,5a974be7410cf7000162e8af,By HELENE COOPER and ERIC SCHMITT,article,U.S. Advances Military Plans for North Korea,"['United States Defense and Military Forces', ...",1,Washington,11,2018-03-01 00:40:01,Asia Pacific,The American military is looking at everything...,The New York Times,News,https://www.nytimes.com/2018/02/28/world/asia/...,1215
2,5a9752a2410cf7000162e8ba,By THE EDITORIAL BOARD,article,Mr. Trump and the ‘Very Bad Judge’,"['Trump, Donald J', 'Curiel, Gonzalo P', 'Unit...",1,Editorial,26,2018-03-01 01:08:46,Unknown,Can you guess which man is the model public se...,The New York Times,Editorial,https://www.nytimes.com/2018/02/28/opinion/tru...,1043
3,5a975310410cf7000162e8bd,By JAVIER C. HERNÁNDEZ,article,"To Erase Dissent, China Bans Pooh Bear and ‘N’","['China', 'Xi Jinping', 'Term Limits (Politica...",1,Foreign,1,2018-03-01 01:10:35,Asia Pacific,Censors swung into action after Mr. Xi’s bid t...,The New York Times,News,https://www.nytimes.com/2018/02/28/world/asia/...,1315
4,5a975406410cf7000162e8c3,"By JESSE DRUCKER, KATE KELLY and BEN PROTESS",article,Loans Flowed to Kushner Cos. After Visits to t...,"['Kushner, Jared', 'Kushner Cos', 'United Stat...",1,Business,1,2018-03-01 01:14:41,Unknown,"Apollo, the private equity firm, and Citigroup...",The New York Times,News,https://www.nytimes.com/2018/02/28/business/ja...,1566


### **Preparação do texto**
Na etapa de preparação do conjunto de dados, primeiro realizaremos a limpeza do texto dos dados, que inclui a remoção de pontuações e letras minúsculas em todas as palavras.

In [ ]:
def Limpeza_Texto(Texto):
  '''
  Função limpar o texto
  - Remove a pontuação do texto
  - Converte o texto para minúsculas
  - Remove caracteres não ASCII
  '''
  # Remoção de Pontuação e Conversão para Minúsculas
  Texto = ''.join( Loop for Loop in Texto if Loop not in string.punctuation).lower()

  # Codificação UTF-8 e Decodificação ASCII
  Texto = Texto.encode('utf8').decode('ascii','ignore')

  return Texto

# Aplicando a função em uma lista
Lista_Textos = [ Limpeza_Texto(Loop) for Loop in Machetes_Localizadas ]
Lista_Textos[:5]

['virtual coins real resources',
 'us advances military plans for north korea',
 'mr trump and the very bad judge',
 'to erase dissent china bans pooh bear and n',
 'loans flowed to kushner cos after visits to the white house']

In [ ]:
len(Lista_Textos)

1250

### **Gerando Sequência de Tokens N-gram**

Gerar sequências de tokens N-gram é um processo utilizado em processamento de linguagem natural para representar texto sequencialmente. Um N-grama refere-se a uma sequência contígua de N itens, que no contexto de processamento de texto são tokens individuais (palavras ou caracteres).

Por exemplo, em um texto "O gato pulou", os 2-gramas seriam "O gato", "gato pulou", e assim por diante. Isso captura a ordem das palavras e pode ser usado para entender padrões de linguagem, como coocorrências e dependências locais entre tokens adjacentes.

Esse método é amplamente utilizado em tarefas como modelagem de linguagem, tradução automática e análise de sentimento, ajudando a capturar informações contextuais importantes para análise textual.







In [27]:
# Inicializa um objeto Tokenizer para a tokenização do texto
tokenizer = Tokenizer()

def obter_sequencia_tokens(corpus):
    '''
    Esta função realiza a tokenização de um corpus de texto e gera sequências de tokens (n-gramas).

    Passos que a função executa:
    - Ajusta um objeto Tokenizer aos textos do corpus para construir um índice de palavras.
    - Converte cada linha do corpus em uma lista de tokens, onde cada palavra é substituída pelo seu índice correspondente.
    - Gera todas as subsequências possíveis (n-gramas) para cada linha do corpus.
    - Retorna uma lista de todas as subsequências de tokens e o número total de palavras únicas no corpus.

    Parâmetros:
    - corpus: uma lista de strings, onde cada string é uma linha de texto do corpus.

    Retorna:
    - input_sequences: uma lista de listas, onde cada sublista é uma sequência de tokens.
    - total_words: um inteiro representando o número total de palavras únicas no corpus mais um.
    '''

    # Ajusta o Tokenizer aos textos do corpus, construindo o índice de palavras
    tokenizer.fit_on_texts(corpus)

    # Calcula o número total de palavras no índice + 1 (para considerar o índice 0)
    total_palavras = len(tokenizer.word_index) + 1

    ## convert data to sequence of tokens
    # Inicializa uma lista para armazenar as sequências de tokens
    entrada_sequencia = []

    # Itera sobre cada linha no corpus
    for linha in corpus:

        # Converte a linha em uma sequência de tokens
        token_list = tokenizer.texts_to_sequences( [linha] )[0]

        # Cria n-gramas a partir da sequência de tokens
        for Loop in range(1, len(token_list)):

          # Gera um n-grama que inclui do primeiro token até o token Loop+1
          n_gram_sequence = token_list[:Loop+1]

          # Adiciona a sequência de tokens ao conjunto de entrada
          entrada_sequencia.append(n_gram_sequence)

    # Retorna as sequências de entrada e o número total de palavras
    return entrada_sequencia, total_palavras

# Chama a função e armazena as sequências de entrada e o total de palavras
sequencia_entrada, total_palavras = obter_sequencia_tokens(Lista_Textos)

# Exibe as primeiras 10 sequências de entrada
sequencia_entrada[:12]

[[1119, 1120],
 [1119, 1120, 116],
 [1119, 1120, 116, 1121],
 [31, 1122],
 [31, 1122, 589],
 [31, 1122, 589, 392],
 [31, 1122, 589, 392, 7],
 [31, 1122, 589, 392, 7, 61],
 [31, 1122, 589, 392, 7, 61, 70],
 [117, 10],
 [117, 10, 6],
 [117, 10, 6, 1]]

### **Preenchendo as sequências**

Preencher as sequências refere-se ao processo de ajustar o comprimento das sequências de dados para que todas tenham o mesmo tamanho. Isso é importante em modelos de aprendizado profundo, como redes neurais, onde os dados de entrada devem ter dimensões uniformes para serem processados eficientemente.

Geralmente, as sequências são preenchidas com tokens especiais (como zeros) antes ou depois dos dados reais, de modo que todas tenham o mesmo comprimento máximo. Isso facilita o processamento em lotes (batches) e otimiza o desempenho durante o treinamento e a inferência dos modelos.







In [33]:
def gerar_sequencia_preenchida(sequencia_entrada):
    '''
    Esta função padroniza as sequências de entrada para que todas tenham o mesmo comprimento,
    e então as divide em preditores e rótulos para treinamento de um modelo.

    Passos que a função executa:
    - Encontra o comprimento máximo das sequências de entrada.
    - Padroniza (preenche) todas as sequências de entrada para que tenham o mesmo comprimento.
    - Divide as sequências de entrada em preditores (todos os tokens exceto o último) e rótulos (o último token).
    - Converte os rótulos em uma matriz categórica (one-hot encoding).

    Parâmetros:
    - sequencia_entrada: uma lista de listas, onde cada sublista é uma sequência de tokens.

    Retorna:
    - predictors: uma matriz onde cada linha é uma sequência de tokens usada como entrada para o modelo.
    - label: uma matriz categórica onde cada linha é o rótulo correspondente à sequência de entrada.
    - max_sequence_len: um inteiro representando o comprimento máximo das sequências padronizadas.
    '''

    # Encontra o comprimento máximo das sequências
    comprimento_max_sequencia = max( [len(Loop) for Loop in sequencia_entrada] )

    # Padroniza as sequências de entrada para que todas tenham o mesmo comprimento
    sequencia_entrada = np.array(pad_sequences(sequencia_entrada, maxlen=comprimento_max_sequencia, padding='pre'))

    # Divide as sequências de entrada em preditores e rótulos
    preditores, label = sequencia_entrada[:,:-1], sequencia_entrada[:,-1]

    # Converte os rótulos em uma matriz categórica (one-hot encoding)
    label = ku.to_categorical(label, num_classes=total_palavras)

    # Retorna os preditores, rótulos e o comprimento máximo das sequências
    return preditores, label, comprimento_max_sequencia

# Chama a função para gerar as sequências padronizadas, preditores e rótulos
preditores, label, comprimento_max_sequencia = gerar_sequencia_preenchida(sequencia_entrada)

### **Treinamento do Modelo**

In [37]:
def criacao_modelo(max_sequence_len, total_words):
    '''
    Esta função cria e compila um modelo de rede neural sequencial usando Keras, configurado para processamento de sequências de texto.

    Passos que a função executa:
    - Define o comprimento da entrada como o comprimento máximo da sequência menos 1.
    - Inicializa um modelo sequencial.
    - Adiciona uma camada de Embedding para transformar índices de palavras em vetores de dimensão 10.
    - Adiciona uma camada LSTM com 100 unidades para capturar dependências sequenciais nos dados.
    - Adiciona uma camada de Dropout com taxa de 0.1 para regularização.
    - Adiciona uma camada densa (fully connected) com ativação 'softmax' para gerar uma distribuição de probabilidade sobre o vocabulário.
    - Compila o modelo com a função de perda 'categorical_crossentropy' e o otimizador 'adam'.

    Parâmetros:
    - max_sequence_len: um inteiro representando o comprimento máximo das sequências de entrada.
    - total_words: um inteiro representando o tamanho do vocabulário.

    Retorna:
    - model: o modelo de rede neural sequencial compilado.
    '''

    # Define o comprimento da entrada como o comprimento máximo da sequência menos 1
    input_len = max_sequence_len - 1

    # Inicializa um modelo sequencial
    model = Sequential()

    # Adiciona a camada de Embedding de entrada
    model.add(Embedding(total_words, 10, input_length=input_len))

    # Adiciona a camada oculta 1 - Camada LSTM
    model.add(LSTM(100))

    # Adiciona a camada de Dropout para regularização
    model.add(Dropout(0.1))

    # Adiciona a camada de saída
    model.add(Dense(total_words, activation='softmax'))

    # Compila o modelo com função de perda de entropia cruzada categórica e otimizador Adam
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    # Retorna o modelo compilado
    return model

# Cria o modelo usando o comprimento máximo da sequência e o número total de palavras
model = criacao_modelo(comprimento_max_sequencia, total_palavras)

# Exibe o resumo da arquitetura do modelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 17, 10)            35820     
                                                                 
 lstm_1 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 3582)              361782    
                                                                 
Total params: 442002 (1.69 MB)
Trainable params: 442002 (1.69 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**DEMOROU EM TORNO DE 20 MINUTOS**

In [38]:
# Vamos treinar o modelo
# Vai lavar louça, lavar carro, assistir série e depois volta :X
model.fit( preditores, label, epochs=100, verbose=5 )

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

### **Gerando o texto**

In [52]:
def generate_text(texto_inicial, proximas_palavras, model, sequencia_max):
    """
    Esta função gera texto previsível com base em um texto inicial, um número especificado de palavras para gerar,
    um modelo treinado e o comprimento máximo da sequência de entrada.

    Parâmetros:
    - texto_inicial: string, o texto inicial a partir do qual o novo texto será gerado.
    - proximas_palavras: inteiro, o número de palavras a serem geradas.
    - model: o modelo de rede neural treinado usado para prever as próximas palavras.
    - sequencia_max: inteiro, o comprimento máximo das sequências usadas durante o treinamento do modelo.

    Retorna:
    - Uma string contendo o texto inicial seguido pelas novas palavras geradas, com a primeira letra de cada palavra em maiúscula.
    """

    for Loop in range(proximas_palavras):

        # Converte o texto inicial em uma lista de tokens
        lista_token = tokenizer.texts_to_sequences([texto_inicial])[0]

        # Padroniza a lista de tokens para ter o comprimento máximo da sequência menos 1
        lista_token = pad_sequences([lista_token], maxlen=sequencia_max-1, padding='pre')

        # Usa o modelo para prever a próxima palavra na sequência
        predicted_probs = model.predict(lista_token, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)[0]

        # Inicializa a variável para armazenar a palavra prevista
        palavra_saida = ""

        # Encontra a palavra correspondente ao índice previsto
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                palavra_saida = word
                break

        # Adiciona a palavra prevista ao texto inicial
        texto_inicial += " " + palavra_saida

    # Retorna o texto gerado com a primeira letra de cada palavra em maiúscula
    return texto_inicial.title()

### **Demostrando | Prompt**

Lembrando que os dados são de nóticias do New York times meados 2018.

Naquele momento havia muita citação ao Trump nos artigos.

In [55]:
def demonstrar_geracao_texto(prompt, quantidade_palavras, model, max_sequence_len):
    """
    Esta função demonstra a geração de texto usando um modelo treinado.

    Parâmetros:
    - prompt: string, o texto inicial a partir do qual o novo texto será gerado.
    - quantidade_palavras: inteiro, o número de palavras a serem geradas.
    - model: o modelo de rede neural treinado usado para prever as próximas palavras.
    - max_sequence_len: inteiro, o comprimento máximo das sequências usadas durante o treinamento do modelo.

    Retorna:
    - O texto gerado com a primeira letra de cada palavra em maiúscula.
    """
    # Gera o texto usando o modelo
    texto_gerado = generate_text(prompt, quantidade_palavras, model, max_sequence_len)

    # Exibe o texto gerado
    print("Texto Inicial: ", prompt)
    print("Quantidade de Palavras Geradas: ", quantidade_palavras)
    print("Texto Gerado: ", texto_gerado)

In [56]:
# Exemplo de uso
Prompt = 'president trump'
Quantidade_Palavras = 10

demonstrar_geracao_texto( Prompt, Quantidade_Palavras, model, comprimento_max_sequencia )

Texto Inicial:  president trump
Quantidade de Palavras Geradas:  10
Texto Gerado:  President Trump Tv Commentator As Amazon The Suv Is Be A Census


In [58]:
# Exemplo de uso
Prompt = 'New York'
Quantidade_Palavras = 9

demonstrar_geracao_texto( Prompt, Quantidade_Palavras, model, comprimento_max_sequencia )

Texto Inicial:  New York
Quantidade de Palavras Geradas:  9
Texto Gerado:  New York Forgets Its Juvenile Lifers From The Census Treatment Thrills


In [62]:
# Exemplo de uso
Prompt = 'United States'
Quantidade_Palavras = 12

demonstrar_geracao_texto( Prompt, Quantidade_Palavras, model, comprimento_max_sequencia )

Texto Inicial:  United States
Quantidade de Palavras Geradas:  12
Texto Gerado:  United States About Trump Tied To The White House Built In The 80S Is
